In [20]:
import win32com.client as win32  #없으면 라이브러리 설치 필요 
import pandas as pd

hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")
hwp.RegisterModule("FilePathCheckDLL","AutomationModule")
hwp.Open("C:\\workspace\\크롤링\\파일명.hwp") #파일 열기 



True

In [42]:
excel = win32.gencache.EnsureDispatch("Excel.Application")
wb = excel.Workbooks.Add()
ws = wb.Worksheets(1) #엑셀 열고 워크시트 정의 
df = pd.DataFrame()

ctrlcode = hwp.HeadCtrl #문서 최초의 컨트롤 코드 
while ctrlcode != None: #마지막에 가면 none이 되니까 막음 
    if ctrlcode.CtrlID == "tbl": #컨트롤 코드의 아이디가 "표" 라면?
        hwp.SetPosBySet(ctrlcode.GetAnchorPos(0))
        # 여기서  GetAnchorPos(0)는 해당 컨트롤의 위치를 튜플로 반환해줍니다.
        # 그리고 SetPosyByset메서드는 해당 튜플로 캐럿을 옮기는 메서드입니다. 
        # 둘을 같이 사용하면 해당 컨트롤 앞으로 이동하게 됨
        hwp.FindCtrl()#컨트롤코드 앞에서 "선택" 
        hwp.Run("ShapeObjTableSelcell")#선택한 부호가 표라면 A1셀을 선택한 상태가 됨 
        hwp.Run("TableCellBlockExtend")
        hwp.Run("TableCellBlockExtend")#F5를 두 번 눌려서 셀 전체 선택
        # 표 바깥에서 표를 선택하지 않고 셀 전체를 번거롭게 선택하는 이유는 
        #혹여, "글자처럼 취급"이 해체되어 있는 표가 있을까봐 그렇습니다. 
        #글자처럼 취급되지 않은 표는 표 바깥에서 표를 선택하기가 까다로움
        hwp.Run("Copy") #클립보드로 복사합니다. 
        excel.ActiveSheet.Paste() #현재 엑셀에 열린시트에 클립보드 내용을 붙여넣기  
        excel.Selection.Copy() #붙여넣은 엑셀시트 범위를 그대로 복사함 
        ws.Range("A1").Select() #이 코드를 넣어서 선택을 초기화하지 않으면 
        # 다음표를 붙어 넣었을때 Copy()의 범위가 기존 범위에서 안바뀌더라고요. 
        # 첫 번째 표 사이즈로만 계속 복사되버려서, 선택초기화를 위해 이 라인을 넣었습니다. 
        df = pd.concat([df, pd.read_clipboard()])
    else : 
        pass    
    ctrlcode = ctrlcode.Next
    
    
df.describe()



,항목,의견
count,12,12
unique,4,4
top,전략목표체계,"기존에 있는 전략을 범주화만 다시 하여 재구성한 느낌이긴 하지만, 이원화되어 있던 ..."
freq,3,3


In [43]:
len(df)

12

In [44]:

df.describe()

,항목,의견
count,12,12
unique,4,4
top,전략목표체계,"기존에 있는 전략을 범주화만 다시 하여 재구성한 느낌이긴 하지만, 이원화되어 있던 ..."
freq,3,3


In [46]:
df.to_excel('inventors.xlsx', index = False)